In [18]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from surprise import Dataset, Reader, accuracy, SVD, dump
from surprise.model_selection import GridSearchCV
import pickle

In [19]:
ratings = pd.read_csv("ratings.csv")
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [20]:
movies = pd.read_csv("movies.csv")
movies = movies.drop("genres", axis=1)
movies.head()

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


In [21]:
df = ratings.drop("timestamp", axis=1)
df

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0
...,...,...,...
100831,610,166534,4.0
100832,610,168248,5.0
100833,610,168250,5.0
100834,610,168252,5.0


In [22]:
# using S_train = 80%, S_dev = 10%, S_test = 10%
trainset, testset = train_test_split(df, train_size=0.8)

print("Training set size: ", trainset.shape)


print("Test set size: ", testset.shape)

Training set size:  (80668, 3)
Test set size:  (20168, 3)


In [23]:
reader = Reader(rating_scale = (1, 5))

train_data = Dataset.load_from_df(trainset, reader)
test_data = Dataset.load_from_df(testset, reader)

In [24]:
test_data = test_data.build_full_trainset()

In [25]:
test_data = test_data.build_testset()

In [26]:
final_algo = SVD()

In [27]:
# param_grid = {"n_epochs": [5, 10, 20], "lr_all": [0.002, 0.005], "reg_all": [0.3, 0.4, 0.6], "n_factors":[50, 100, 200]}

# gs = GridSearchCV(SVD, param_grid, measures=["rmse"], cv=10, n_jobs=10, joblib_verbose=15)
# gs.fit(train_data)

In [28]:
# print(gs.best_score["rmse"])


In [29]:
# print(gs.best_params["rmse"])


In [30]:
# final_algo = gs.best_estimator["rmse"]

train_data = train_data.build_full_trainset()
final_algo.fit(train_data)

In [31]:
# Finally, predicting and calculating accuracy on test_data (S_test)
pred_test = final_algo.test(test_data)
accuracy.rmse(pred_test)

RMSE: 0.8771


0.877095707318606

In [32]:
# All data
data = Dataset.load_from_df(df, reader)
data = data.build_full_trainset()

final_algo.fit(data)

In [33]:
pickle.dump(final_algo, open("svd_model1.sav", "wb"))